In [1]:
import sys, subprocess
print("Using Python:", sys.executable)  # should match the kernel you expect
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "pandas", "SQLAlchemy>=2.0", "psycopg2-binary"])
import pandas as pd
from sqlalchemy import create_engine, text
print("✅ packages ready")


Using Python: D:\Nishant\DA\source\python.exe
✅ packages ready


In [2]:
DB_USER = "postgres"
DB_PASS = "Nishant1302"
DB_HOST = "localhost"
DB_PORT = 5432
DB_NAME = "weather"          # <- use your actual DB name (avoid spaces)
TABLE   = "weather_rain_au"  # table you already created

# full path to your CSV (change this!)
CSV_PATH = "weatherAUS_rainfall_prediction_dataset_cleaned.csv"


In [3]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

with engine.connect() as conn:
    version = conn.exec_driver_sql("SELECT version();").scalar()
    print("Connected to:", version)


Connected to: PostgreSQL 17.6 on x86_64-windows, compiled by msvc-19.44.35213, 64-bit


In [4]:
# read the CSV
df = pd.read_csv(CSV_PATH)
print("CSV shape:", df.shape)
df.head(2)


CSV shape: (145460, 23)


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2010-04-30,Adelaide,10.8,21.2,0.0,1.8,6.60,ENE,22.0,ENE,...,66.0,48.0,1028.4,1024.7,4.45,4.51,15.7,21.0,No,No
1,2014-07-22,Adelaide,3.7,19.0,0.0,1.4,7.61,N,24.0,N,...,75.0,32.0,1024.2,1021.1,4.45,4.51,9.6,18.5,No,No


In [5]:
# expected column order (as created in your table)
expected_cols = [
    "Date","Location","MinTemp","MaxTemp","Rainfall","Evaporation","Sunshine",
    "WindGustDir","WindGustSpeed","WindDir9am","WindDir3pm","WindSpeed9am",
    "WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm",
    "Cloud9am","Cloud3pm","Temp9am","Temp3pm","RainToday","RainTomorrow"
]

# make a case-insensitive rename map so columns match exactly
rename_map = {c.lower(): c for c in expected_cols}
df = df.rename(columns={c: rename_map.get(c.lower(), c) for c in df.columns})

# ensure Date is a real date (Postgres DATE type)
if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.date

# keep only the expected columns in the right order
missing = [c for c in expected_cols if c not in df.columns]
extra   = [c for c in df.columns if c not in expected_cols]
print("Missing columns:", missing)
print("Extra columns (will be ignored):", extra)

df = df[[c for c in expected_cols if c in df.columns]]
df.head(2)


Missing columns: []
Extra columns (will be ignored): []


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2010-04-30,Adelaide,10.8,21.2,0.0,1.8,6.60,ENE,22.0,ENE,...,66.0,48.0,1028.4,1024.7,4.45,4.51,15.7,21.0,No,No
1,2014-07-22,Adelaide,3.7,19.0,0.0,1.4,7.61,N,24.0,N,...,75.0,32.0,1024.2,1021.1,4.45,4.51,9.6,18.5,No,No


In [6]:
# convert all column names to lowercase
df.columns = df.columns.str.lower()




In [7]:
# now insert into SQL
df.to_sql(
    TABLE, 
    engine, 
    if_exists="append", 
    index=False, 
    chunksize=10000, 
    method="multi"
)

print("✅ upload complete with lowercase columns")

✅ upload complete with lowercase columns
